In [292]:
import pandas as pd 
import numpy as np
import re

In [293]:
data1 = pd.read_csv("truffle_zzappa_label.csv")
data2 = pd.read_csv("백종원 있는 미역초무침 댓글 2제외.csv")
data3 = pd.read_csv("mara_1-7445.csv")

In [294]:
data = pd.concat([data2, data3], axis = 0)

In [295]:
data1

,Unnamed: 0,Comments,tokens,label
0,0,솔직히 먹어봤는데 트러플을 접해보지 못한 일반인인 나는 차이를 느끼지 못했다,솔직히 먹어 봤는데 트러플 해보지 일반인 차이 느끼지 했다,0
1,2,그냥 짜파게티에 달걀후라이가 젤 맛있음,그냥 짜파게티 달걀후라이 맛있음,0
2,5,저는 짜파게티 맛이 예전맛이 안나는거 같아서 항상 아쉬웠었는데 트러플 소스 들어가고...,짜파게티 안나는거 같아서 항상 아쉬웠었는데 트러플 소스 들어가고 나서 옜날 어릴 먹...,1
3,7,오 컵라면 뚜껑에 젓가락 끼워놓을 수 있는 곳이 있는줄 몰랐네요ㅋㅋㅋ 완전 꿀팁...,컵라면 뚜껑 젓가락 끼워놓을 있는 있는줄 몰랐네요 완전 꿀팁 트러플 오일 제조 송로...,0
4,10,솔직한 리뷰 잘 봤구요 그냥 트러플 오일 하나 사서 두어 방울 뿌려먹는게 낫겠지 ...,솔직한 리뷰 봤구요 그냥 트러플 오일 하나 사서 두어 방울 뿌려 먹는게 낫겠지 싶네...,0
...,...,...,...,...
377,2192,드셨던 짜파게티 중에 최고라니 저두 당장 먹어봐야겠어요,드셨던 짜파게티 최고 당장 먹어 봐야겠어요,1
378,2195,맛있겠다,맛있겠다,1
379,2213,맜있겠다 나짜장면 좋아하는데,있겠다 짜장면 좋아하는데,1
380,2226,진짜 맛있겠다 못참아,진짜 맛있겠다 참아,1


In [296]:
test_df = data
test_df = test_df.replace(np.nan, '', regex=True)
test_df.head()

,Unnamed: 0,Unnamed: 0.1,Comments,tokens,label
0,640,15,라면의품격을 한차원 올렸네요 잘먹고갑니다,"['라면', '품격', '을', '차원', '올리다', '먹다', '가다']",1
1,643,18,미역초비빔면 괜찮아보이네요 ㅎㅎ 원래 오이를 채썰어서 비빔면이랑 같이 비벼서 많이 ...,"['미역', '초', '비빔면', '괜찮다', '보이다', 'ㅎㅎ', '원래', '...",1
2,644,19,오이오이오이 오늘도 단순하지만 영양가는 꼭 들어가는 한끼이네요 ㅎㅎㅎㅎㅎ 저 까만것...,"['오이오이오이', '오늘', '단순하다', '영양가', '꼭', '들어가다', '...",1
3,645,20,미역초 비빔면 정말 맛있게 드시네요 저도 한입 먹고싶어요 풀시청하고 갑니다,"['미역', '초', '비빔면', '정말', '맛있다', '들다', '저', '입'...",1
4,647,22,노다이어트ㅋㅋㅋ 근데 진짜로 건강음식 같네요 먹고싶,"['노', '다이어트', 'ㅋㅋㅋ', '근데', '진짜', '로', '건강', '음...",1


In [297]:
def preprocess(text) :
    text = text.replace('[','')
    text = text.replace(']','')
    return text

In [298]:
test_df['Comments'] = test_df['Comments'].map(lambda x : preprocess(x))

In [299]:
data2['Comments'] = data2['Comments'].map(lambda x : preprocess(x))

In [300]:
import konlpy.tag
from konlpy.tag import Mecab
mecab = Mecab()

#mecab으로 토큰화하고 한 글자는 젝하고, stop_word도 제거하자

stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']

# 이거는 한글자 빼고 / stopword 랑 stopPos 뺴는거. 이따가 쓸 것임. text는 꼭 str
def token_mecab(text) :
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])> 1]
    text = [i for i in text if i[0] not in stop_word] 
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [301]:
def make_tokens(df) :
    df['tokens'] = ''
    tokens_list = []
    for i, row in df.iterrows() :
        if i %10000 == 0 :
            print(i, '/', len(df))
        token = token_mecab(df['Comments'][i])
        df['tokens'][i] = ' '.join(token)
        
    return df

In [302]:
data = data[['Comments', 'label']]
data2 = data2[['Comments', 'label']]

In [303]:
data =  data.fillna(2)
data2 =  data2.fillna(2)

In [304]:
data2.isnull().sum()

Comments    0
label       0
dtype: int64

In [305]:
# 트레인!
X_train = list(data['Comments'])
Y_train = list(data['label'])

# 이거 아님 주의 !!
# X_train = list(df['document'])
# Y_train = list(df['label'])

In [306]:
test_df

,Unnamed: 0,Unnamed: 0.1,Comments,tokens,label
0,640,15,라면의품격을 한차원 올렸네요 잘먹고갑니다,"['라면', '품격', '을', '차원', '올리다', '먹다', '가다']",1
1,643,18,미역초비빔면 괜찮아보이네요 ㅎㅎ 원래 오이를 채썰어서 비빔면이랑 같이 비벼서 많이 ...,"['미역', '초', '비빔면', '괜찮다', '보이다', 'ㅎㅎ', '원래', '...",1
2,644,19,오이오이오이 오늘도 단순하지만 영양가는 꼭 들어가는 한끼이네요 ㅎㅎㅎㅎㅎ 저 까만것...,"['오이오이오이', '오늘', '단순하다', '영양가', '꼭', '들어가다', '...",1
3,645,20,미역초 비빔면 정말 맛있게 드시네요 저도 한입 먹고싶어요 풀시청하고 갑니다,"['미역', '초', '비빔면', '정말', '맛있다', '들다', '저', '입'...",1
4,647,22,노다이어트ㅋㅋㅋ 근데 진짜로 건강음식 같네요 먹고싶,"['노', '다이어트', 'ㅋㅋㅋ', '근데', '진짜', '로', '건강', '음...",1
...,...,...,...,...,...
2993,2993,,젓가락 질 하는 것도 욕하는건가 그래서 저가 쓰시는 그면 진짜 제이제이님 인...,,2
2994,2994,,탄탄멘은 국물빼고 비벼먹는거 아닌가유,,2
2995,2995,,일본거는 시국이라서 안가져오신건가,,2
2996,2996,,인도네시아 미고랭 현지호텔이나 식당가서 먹으면 진짜 존맛탱이에요 반숙계란이 국...,,2


In [307]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [308]:
data

,Comments,label
0,라면의품격을 한차원 올렸네요 잘먹고갑니다,1.0
1,미역초비빔면 괜찮아보이네요 ㅎㅎ 원래 오이를 채썰어서 비빔면이랑 같이 비벼서 많이 ...,1.0
2,오이오이오이 오늘도 단순하지만 영양가는 꼭 들어가는 한끼이네요 ㅎㅎㅎㅎㅎ 저 까만것...,1.0
3,미역초 비빔면 정말 맛있게 드시네요 저도 한입 먹고싶어요 풀시청하고 갑니다,1.0
4,노다이어트ㅋㅋㅋ 근데 진짜로 건강음식 같네요 먹고싶,1.0
...,...,...
2993,젓가락 질 하는 것도 욕하는건가 그래서 저가 쓰시는 그면 진짜 제이제이님 인...,2.0
2994,탄탄멘은 국물빼고 비벼먹는거 아닌가유,2.0
2995,일본거는 시국이라서 안가져오신건가,2.0
2996,인도네시아 미고랭 현지호텔이나 식당가서 먹으면 진짜 존맛탱이에요 반숙계란이 국...,2.0


In [309]:
data2

,Comments,label
0,라면의품격을 한차원 올렸네요 잘먹고갑니다,1.0
1,미역초비빔면 괜찮아보이네요 ㅎㅎ 원래 오이를 채썰어서 비빔면이랑 같이 비벼서 많이 ...,1.0
2,오이오이오이 오늘도 단순하지만 영양가는 꼭 들어가는 한끼이네요 ㅎㅎㅎㅎㅎ 저 까만것...,1.0
3,미역초 비빔면 정말 맛있게 드시네요 저도 한입 먹고싶어요 풀시청하고 갑니다,1.0
4,노다이어트ㅋㅋㅋ 근데 진짜로 건강음식 같네요 먹고싶,1.0
...,...,...
528,미역이 많이 들어 있군요 저도 신제품 나오면 바로 사서 먹어 보는데 이 제품은 아...,1.0
529,꼭 한국에 가서 사먹을거야,1.0
530,맛있는 면,1.0
531,초장비빔면 너무 맛있어 보여요 차순영님 방에서 찾아와 친구맺고 갑니다 승승장...,1.0


In [310]:
predic_label = []
for i, row in data1.iterrows():
    predic = clf.predict(count_vect.transform([data1['Comments'][i]]))[0]
    predic_label.append(predic)
data1['predic_label'] = predic_label
data1.head(10)

,Unnamed: 0,Comments,tokens,label,predic_label
0,0,솔직히 먹어봤는데 트러플을 접해보지 못한 일반인인 나는 차이를 느끼지 못했다,솔직히 먹어 봤는데 트러플 해보지 일반인 차이 느끼지 했다,0,2.0
1,2,그냥 짜파게티에 달걀후라이가 젤 맛있음,그냥 짜파게티 달걀후라이 맛있음,0,2.0
2,5,저는 짜파게티 맛이 예전맛이 안나는거 같아서 항상 아쉬웠었는데 트러플 소스 들어가고...,짜파게티 안나는거 같아서 항상 아쉬웠었는데 트러플 소스 들어가고 나서 옜날 어릴 먹...,1,2.0
3,7,오 컵라면 뚜껑에 젓가락 끼워놓을 수 있는 곳이 있는줄 몰랐네요ㅋㅋㅋ 완전 꿀팁...,컵라면 뚜껑 젓가락 끼워놓을 있는 있는줄 몰랐네요 완전 꿀팁 트러플 오일 제조 송로...,0,2.0
4,10,솔직한 리뷰 잘 봤구요 그냥 트러플 오일 하나 사서 두어 방울 뿌려먹는게 낫겠지 ...,솔직한 리뷰 봤구요 그냥 트러플 오일 하나 사서 두어 방울 뿌려 먹는게 낫겠지 싶네...,0,2.0
5,11,먹어봤는데 그냥 호기심에 한번 먹어봄직한 맛이고 두번은 안 사먹을 맛임,먹어 봤는데 그냥 호기심 한번 먹어 직한 번은 사먹을 맛임,0,0.0
6,12,진짜 트러플이 무슨맛인지는 모르겠지만 난 개인적으로 맛있음,진짜 트러플 무슨 모르겠지만 개인 맛있음,1,2.0
7,14,짜파게티와 첫 키스,짜파게티 키스,1,2.0
8,15,진짜 표정에서 아쉬움이 엄청 묻어나오시네요ㅋㅋ,진짜 표정 아쉬움 묻어 나오시네요,0,2.0
9,17,오일도 저렴이니 트러플향도 그닥인데 그나마 있던향도 짜장에 묻혀 일반이랑 큰...,오일도 저렴 트러플 그닥 있던 짜장 묻혀 일반 차이 없드라구,0,2.0


In [313]:
for i in range(len(data1)) :
    data1['predic_label'][i] = int(data1['predic_label'][i])

C:\anaconda3\envs\sjpark\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [314]:

# 계산하는 방법임
# predic 과 실제 label을 비교하여 정의대로 값을 뽑아낸다.
TP = FN = FP = TN = other = 0
for i in range(len(data1)):
    if data1['label'][i] == 1 and data1['predic_label'][i] == 1:
        TP +=1
    elif data1['label'][i] == 0 and data1['predic_label'][i] == 0:
        TN +=1
    elif data1['label'][i] == 0 and data1['predic_label'][i] == 1:
        FN +=1
        print(i)
    elif data1['label'][i] == 1 and data1['predic_label'][i] == 0:
        FP +=1
        print(i)
    else:
        other +=1
        
print(TP,TN,FN,FP)

49
52
54
69
72
76
86
109
121
127
136
149
307
310
314
333
76 6 15 1


In [315]:
ACC = (TP+TN)/(TP+TN+FP+FN)
print('ACC: ',ACC)

ACC:  0.8367346938775511


In [329]:
data1.loc[49]

Unnamed: 0                           84
Comments        크러플소스를 따로 사서 뿌려먹는게 갑입니가
tokens              크러 소스 사서 뿌려 먹는게 입니가
label                                 0
predic_label                          1
Name: 49, dtype: object

In [330]:
data1.loc[52]

Unnamed: 0                                                     92
Comments        만약 이게 트러플향이라면 난 다시는 트러플 안먹을래 했는데 그게 진짜 향이었다니  ...
tokens          만약 트러플 라면 다시는 트러플 먹을래 했는데 그게 진짜 이었다니 시큼시큼하면서 찌...
label                                                           0
predic_label                                                    1
Name: 52, dtype: object

In [331]:
data1.loc[54]

Unnamed: 0                                          98
Comments        기존 짜파게티가 훨씬 맛있습니다 다신 안사먹을라고요 드럽게 맛 없네요
tokens                 기존 짜파게티 맛있습니다 다신 사먹을라고요 드럽게 없네요
label                                                0
predic_label                                         1
Name: 54, dtype: object

In [332]:
data1.loc[69]

Unnamed: 0                          133
Comments        농심 제품은 불매라 먹어보진 않을듯 합니다
tokens           농심 제품 불매 먹어 보진 않을듯 합니다
label                                 0
predic_label                          1
Name: 69, dtype: object

In [333]:
data1.loc[72]

Unnamed: 0                       137
Comments        농심 전범기업이랑 손잡았다고 나오던데
tokens           농심 전범 기업 손잡았다고 나오던데
label                              0
predic_label                       1
Name: 72, dtype: object

In [334]:
data1.loc[86]

Unnamed: 0                                                    163
Comments        고물님 후기 기다렸어요  아직 주변 마트에서 못봤는데    그냥 트러플 오일 사서 ...
tokens                    고물 후기 기다렸어요 주변 마트 봤는데 그냥 트러플 오일 사서 먹을래요
label                                                           0
predic_label                                                    1
Name: 86, dtype: object

In [335]:
data1.loc[109]

Unnamed: 0           207
Comments        농심   왜그래
tokens             농심 그래
label                  0
predic_label           1
Name: 109, dtype: object

In [336]:
data1.loc[121]

Unnamed: 0                               222
Comments        오히려 짜파게티에다 트러플오일 넣는게 더 괜찮을지도
tokens             오히려 짜파게티 트러플 오일 넣는게 괜찮을지도
label                                      0
predic_label                               1
Name: 121, dtype: object

In [337]:
data1.loc[127]

Unnamed: 0                                                    234
Comments        농심 설문조사 결과로 나온 제품인것 같더라구요 저도 투표 했구요 헣  저같이 트러플...
tokens          농심 설문조사 결과 나온 제품 같더라구요 투표 했구요 트러플 오일이 익숙하지 않은 ...
label                                                           0
predic_label                                                    1
Name: 127, dtype: object